In [11]:
import datetime

from util.util import importstr
from util.logconf import logging
log = logging.getLogger('nb')

In [13]:
def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')  # <1>
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))  # <2>
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}).main()".format(app, argv))

In [14]:
import os
import shutil

# clean up any old data that might be around.
# We don't call this by default because it's destructive, 
# and would waste a lot of time if it ran when nothing 
# on the application side had changed.
def cleanCache():
    shutil.rmtree('data-unversioned/cache')
    os.mkdir('data-unversioned/cache')

# cleanCache()

In [15]:
training_epochs = 20
experiment_epochs = 10
final_epochs = 50

training_epochs = 2
experiment_epochs = 2
final_epochs = 5
seg_epochs = 10

## Chapter 11

In [ ]:
run('p2ch11.prepcache.LunaPrepCacheApp')

In [17]:
run('p2ch11.training.LunaTrainingApp', '--epochs=1')

2021-07-06 02:59:50,690 INFO     pid:3212 nb:004:run Running: p2ch11.training.LunaTrainingApp(['--num-workers=4', '--epochs=1']).main()
2021-07-06 03:00:15,381 INFO     pid:3212 p2ch11.training:138:main Starting LunaTrainingApp, Namespace(batch_size=32, comment='dwlpt', epochs=1, num_workers=4, tb_prefix='p2ch11')
2021-07-06 03:00:17,943 INFO     pid:3212 p2ch11.dsets:185:__init__ <p2ch11.dsets.LunaDataset object at 0x00000285852C6630>: 51244 training samples
2021-07-06 03:00:17,960 INFO     pid:3212 p2ch11.dsets:185:__init__ <p2ch11.dsets.LunaDataset object at 0x000002858AC531D0>: 5694 validation samples
2021-07-06 03:00:17,962 INFO     pid:3212 p2ch11.training:151:main Epoch 1 of 1, 1602/178 batches of size 32*1
2021-07-06 03:00:18,062 WARNING  pid:3212 util.util:221:enumerateWithEstimate E1 Training ----/1602, starting


In [ ]:
run('p2ch11.training.LunaTrainingApp', f'--epochs={experiment_epochs}')

## Chapter 12

In [ ]:
run('p2ch12.prepcache.LunaPrepCacheApp')

In [ ]:
run('p2ch12.training.LunaTrainingApp', '--epochs=1', 'unbalanced')

In [ ]:
run('p2ch12.training.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', 'balanced')

In [ ]:
run('p2ch12.training.LunaTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-flip', 'flip')

In [ ]:
run('p2ch12.training.LunaTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-offset', 'offset')

In [ ]:
run('p2ch12.training.LunaTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-scale', 'scale')

In [ ]:
run('p2ch12.training.LunaTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-rotate', 'rotate')

In [ ]:
run('p2ch12.training.LunaTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-noise', 'noise')

In [ ]:
run('p2ch12.training.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augmented', 'fully-augmented')

## Chapter 13

In [ ]:
run('p2ch13.prepcache.LunaPrepCacheApp')

In [ ]:
run('p2ch13.training.LunaTrainingApp', f'--epochs={final_epochs}', '--balanced', '--augmented', 'final-cls')

In [ ]:
run('p2ch13.train_seg.LunaTrainingApp', f'--epochs={seg_epochs}', '--augmented', 'final-seg')

## Chapter 14

In [ ]:
run('p2ch14.prepcache.LunaPrepCacheApp')

In [ ]:
run('p2ch14.training.ClassificationTrainingApp', f'--epochs=100', 'nodule-nonnodule')

In [ ]:
run('p2ch14.training.ClassificationTrainingApp', f'--epochs=40', '--malignant', '--dataset=MalignantLunaDataset',
    '--finetune=''data/part2/models/cls_2020-02-06_14.16.55_final-nodule-nonnodule.best.state',
    'finetune-head')


In [ ]:
run('p2ch14.training.ClassificationTrainingApp', f'--epochs=40', '--malignant', '--dataset=MalignantLunaDataset',
    '--finetune=''data/part2/models/cls_2020-02-06_14.16.55_final-nodule-nonnodule.best.state',
    '--finetune-depth=2',
    'finetune-depth2')


In [ ]:
run('p2ch14.nodule_analysis.NoduleAnalysisApp', '--run-validation')

In [ ]:
run('p2ch14.nodule_analysis.NoduleAnalysisApp', '--run-validation', '--malignancy-path')

# mycode

In [28]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
d = np.random.rand(50,1)

class mdset(Dataset):
    def __init__(self, dt):
        self.version = '1.2.3'
        self.dt = dt
    def __getitem__(self, index):
        return (index, self.dt[index])

    def __len__(self) -> int:
        return len(self.dt)

dl = DataLoader(
    mdset(d),
    batch_size=4,
    num_workers=0,
    pin_memory = False
)

m = iter(dl)
n = next(m)
print(n)

[tensor([0, 1, 2, 3]), tensor([[0.0891],
        [0.1210],
        [0.6369],
        [0.4319]], dtype=torch.float64)]


In [9]:
import torch.nn as nn
import torch
m0 = nn.Softmax(dim=0)
m1 = nn.Softmax(dim=1)
input = torch.randn(2, 3)
print(input)

print(m0(input))
print(m1(input))

tensor([[ 1.2829,  2.5062, -1.9215],
        [-0.7234, -0.9330,  0.5601]])
tensor([[0.8815, 0.9689, 0.0772],
        [0.1185, 0.0311, 0.9228]])
tensor([[0.2253, 0.7656, 0.0091],
        [0.1845, 0.1496, 0.6659]])
